# Подключение

In [1]:
import pyodbc

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()

## Pandas

In [3]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [5]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

## Необходимые функции

Позволяет получить список характеристик для всех предложений, указанных в list_of_keys. Предложения указываются в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

Преобразование из байтов в такую строку делается следующим образом:
```python
# type(x) == bytes
result = '0x' + x.hex().upper()
```

In [6]:
def get_harks_by_object_keys(task_id):
    """
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 01 THEN 
        CASE
            WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
            WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
            ELSE 'НетДанных'
        END
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача = (SELECT _IDRRef FROM [rway].[dbo].[_Task62] WHERE _FLD198 = '{}')
    '''.format(task_id)
    
    cursor.execute(query)
    
    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), cursor.fetchall()))
    data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
    # Создание словаря для DataFrame
    return data_df

    

Функция позволяет получить все предложения по конкретной задаче. Задача указывается в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

In [7]:
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задачи
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {}
          Код,
          Ссылка,
          АдресAhunter,
          АктуальнаяСсылкаИсточника,
          ДатаПересмотраЭкспозиции,
          ДатаПроверкиАктуальности,
          ДатаРазмещения,
          Город,
          Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] = (SELECT _IDRRef FROM [rway].[dbo].[_Task62] WHERE _FLD198 = '{}')

    '''.format(top, task_id)
    
    cursor.execute(query)
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
    df.columns = list(map(lambda x: x[0], cursor.description))
    return df
    

# Боевой запуск

In [8]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0405-0001')

In [58]:
base_df



,Код,Ссылка,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки
0,190726W00152851,0x800014CB330FA3514F01117219D48F16,"обл Тульская, г Тула, пр-кт Ленина, дом 112Б",,2019-07-26 00:00:00,2019-07-26 11:46:39,2019-07-17 00:00:00,Тула,"Предлагаем в аренду помещение, под любой вид б...",ПСН,Коммерческая Недвижимость,Тульская обл,https://tula.cian.ru/rent/commercial/167207871/,Вторичный,Аренда
1,190724W00000832,0x800068D73042759E44B0A57B6B440BD9,"Респ Татарстан, г Казань, ул Чистопольская, до...",,2019-07-24 00:00:00,2019-07-24 14:28:22,2019-07-22 00:00:00,Казань,Арт. 10139303 Уникальное расположение! Продаё...,офисный,Коммерческая Недвижимость,Татарстан Респ,https://kazan.cian.ru/sale/commercial/208565976/,Вторичный,Продажа
2,190725W00692765,0x8000B040883C64044221ED1E7906407E,"обл Московская, г Химки, ул Молодежная, дом 54",,2019-07-25 00:00:00,2019-07-25 18:42:46,2019-07-25 00:00:00,Химки,Номер объекта 71024.,ПСН,Коммерческая Недвижимость,Московская обл,https://khimki.cian.ru/sale/commercial/206520716/,Вторичный,Продажа
3,190724W00022065,0x8000EA404BBC2FA540DF95BAB1C94518,"г Москва, ул Пироговская М., дом 14",,2019-07-24 00:00:00,2019-07-24 21:47:06,2019-07-24 00:00:00,Москва,Предлагаем в долгосрочную прямую аренду офисно...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/212292251/,Вторичный,Аренда
4,190724W00014544,0x8000FEDE7DDC5F2A4D2890E6F8798F75,"обл Кировская, г Киров, ул Заводская, дом 17",,2019-07-24 00:00:00,2019-07-24 19:21:02,2019-07-24 00:00:00,Киров,Арт. 8479574 Продается оздоровительный комплек...,гостиничный,Коммерческая Недвижимость,Кировская обл,https://kirov.cian.ru/sale/commercial/212003814/,Вторичный,Продажа
5,190726W00004545,0x80015483CA4671DB4E53EF1ADDFE88A5,"г Москва, ул Яблочкова, дом 21, корп 3",,2019-07-26 00:00:00,2019-07-26 00:10:05,2019-07-24 00:00:00,Москва,Сдается офисное с мебелью в пешей доступности ...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/207288709/,Вторичный,Аренда
6,190725W00293416,0x800163B6F694D4B74BB3AB9B9003E3B6,"г Москва, пер Волков, дом 7-9, стр 2",,2019-07-25 00:00:00,2019-07-25 16:48:29,2019-07-25 00:00:00,Москва,Готовый арендный бизнес. Помещение свободного ...,ПСН,Коммерческая Недвижимость,Москва г,https://www.cian.ru/sale/commercial/209554117/,Вторичный,Продажа
7,190725W00013365,0x80017E221C64D51643AB29BBB247E2E1,"Респ Башкортостан, г Уфа, ул Менделеева, дом 221",,2019-07-25 00:00:00,2019-07-25 10:57:38,2019-07-24 00:00:00,Уфа,= ОТ СОБСТВЕННИКА. БЕЗ КОМИССИИ = Сдается офис...,офисный,Коммерческая Недвижимость,Башкортостан Респ,https://ufa.cian.ru/rent/commercial/205017169/,Вторичный,Аренда
8,190725W00016745,0x8001BF97A5B023C44F84D27629C9A091,"обл Тюменская, г Тюмень, ул Дзержинского, дом ...",,2019-07-25 00:00:00,2019-07-25 13:36:17,2019-07-25 00:00:00,Тюмень,Сдается в аренду от собственника! Без комиссии...,офисный,Коммерческая Недвижимость,Тюменская обл,https://tyumen.cian.ru/rent/commercial/210943889/,Вторичный,Аренда
9,190724W00015278,0x8001E92DB5321FDF47218ABCCF48A1F4,"г Москва, ул Семёновская Б., дом 40",,2019-07-24 00:00:00,2019-07-24 19:41:21,2019-07-24 00:00:00,Москва,!ПРЯМАЯ АРЕНДА ОТ СОБСТВЕННИКА без комиссии и ...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/212883225/,Вторичный,Аренда


In [9]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys('0001-0405-0001')

In [10]:
har_df

,Наименование,Значение,Ссылка
0,3.Муниципальное Образование,Щербинка,0x949EA1D0521416D3496903B61946CB67
1,3.Муниципальное Образование,район Коптево,0x94C82094B2FCC792431D0FB443363B54
2,3.Муниципальное Образование,,0x950D3646BB34A20E4B51496B92EA2E0E
3,3.Муниципальное Образование,район Западное Дегунино,0x952557AF41F4F7E545901E155D83F7FF
4,3.Муниципальное Образование,город Казань,0x9593C145FDECBCC0443112162CCB3E59
5,3.Муниципальное Образование,город Казань,0x95B08CA4660A46464F0D45B5A21C9D0C
6,3.Муниципальное Образование,город Чебоксары,0x95F8DC961A33A6B84DA3D75688585945
7,3.Муниципальное Образование,Одинцовский,0x9609F9A1F50661AC46B41D02B54DD0E0
8,3.Муниципальное Образование,город Владимир,0x964C817538FD9E3C40E1BA071271DAAE
9,3.Муниципальное Образование,район Тверской,0x96AC9C57535E54AA4FF51F5197EE23C5


In [40]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='outer')

In [41]:
result_df

,Код,Ссылка,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Дата Сбора Информации,Заголовок,Класс Объекта,Количество Этажей,Назначение Объекта Предложения,Общая Площадь Предложения,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Способ Реализации,Телефон Продавца,Тип объекта недвижимости,Цена,Цена Предложения За 1 кв.м.,Этаж
0,190725W00140365,0x88E88917402AE1934A6D5C6B5B4C1F53,"обл Иркутская, г Иркутск, ул Булавина, дом 10/2",,2019-07-25 00:00:00,2019-07-25 15:25:32,2019-07-19 00:00:00,Иркутск,"Продаем нежилое помещение 341,4м2 на ул.Булави...",торговый,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/54701/,Вторичный,Продажа,город Иркутск,Свердловский,Булавина ул,дом № 10/2,2019-07-25 15:25:32,"Коммерческая недвижимость Помещение, Иркутск, ...",NaN,9.000000,магазин,341.400000,Попадает в полигон,Помещение,"Тысячелетие, Агентство Недвижимости (Ооо Тысяч...",кв.м.,руб.,Без Проведения Торгов,"503-560, 533-333",None,13000000.000000,38078.500293,1
1,190725W00140360,0x8CB5BE84036999684BFF26F492D7D7DD,,,2019-07-25 00:00:00,2019-07-25 15:27:39,2019-07-22 00:00:00,,"Аршан, улица Вересова 83, продается База отдых...",None,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/55022/,Вторичный,Продажа,NaN,NaN,NaN,NaN,2019-07-25 15:27:39,"Коммерческая недвижимость Бизнес, Другое Ирк. ...",NaN,0.000000,None,1826.000000,NaN,None,CENTURY 21 Недвижимость мира,кв.м.,руб.,Без Проведения Торгов,"ул. Чкалова, 15, 6 этаж, тел.: 43-03-64 ул. Ро...",None,8500000.000000,NaN,
2,190725W00140372,0x9151C3BF958E9A8C4CD2F47B4CE456B2,"обл Иркутская, г Иркутск, пер МОПРА, дом 3",,2019-07-25 00:00:00,2019-07-25 15:28:19,2019-07-22 00:00:00,Иркутск,"Иркутск город, переулок МОПРа 3, продается Неж...",ПСН,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/55023/,Вторичный,Продажа,NaN,NaN,МОПРА пер,дом № 3,2019-07-25 15:28:19,"Коммерческая недвижимость Помещение, Иркутск, ...",NaN,0.000000,свободное,89.100000,NaN,Помещение,Century 21 Недвижимость Мира,кв.м.,руб.,Без Проведения Торгов,"ул. Чкалова, 15, 6 этаж, тел.: 43-03-64 ул. Ро...",None,8000000.000000,89786.756453,1
3,190725W00140361,0x9A97E73A826CACD24B700767D593096A,"обл Иркутская, г Иркутск, тер Горка, дом 5",,2019-07-25 00:00:00,2019-07-25 15:29:12,2019-07-25 00:00:00,Иркутск,"склад арочный, холодный, стены металлопрофиль",производственно-складской,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/55006/,Вторичный,Аренда,NaN,NaN,Горка тер,дом № 5,2019-07-25 15:29:12,"Коммерческая недвижимость Помещение, Иркутск, ...",A,0.000000,склад,399.000000,NaN,Помещение,Александр,кв.м.,руб./мес.,Без Проведения Торгов,нет данных,None,40000.000000,100.250627,
4,190725W00140370,0x9C83A00CD62CFC8F46BD8FF3E33AF5A7,"обл Иркутская, г Иркутск, ул Фурье, дом 10",,2019-07-25 00:00:00,2019-07-25 15:26:40,2019-07-18 00:00:00,Иркутск,"Иркутск город, улица Фурье 10, сдается Магазин...",торговый,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/55014/,Вторичный,Аренда,NaN,NaN,Фурье ул,дом № 10,2019-07-25 15:26:40,"Коммерческая недвижимость Помещение, Иркутск, ...",NaN,4.000000,магазин,175.000000,NaN,Помещение,Century 21 Недвижимость Мира,кв.м.,руб./мес.,Без Проведения Торгов,"ул. Чкалова, 15, 6 этаж, тел.: 43-03-64 ул. Ро...",None,150000.000000,857.142857,1
5,190725W00140359,0xA2A9B1D989B5438A4DECC60B34EEE537,"обл Иркутская, г Иркутск, ул Сурнова, дом 30/2",,2019-07-25 00:00:00,2019-07-25 15:27:39,2019-07-23 00:00:00,Иркутск,"Иркутск город, улица Сурнова 30/2, продается Н...",гостиничный,Коммерческая Недвижимость,Иркутская обл,https://realty.irk.ru/comm/55027/,Вторичный,Продажа,NaN,NaN,Сурнова ул,дом № 30/2,2019-07-25 15:27:39,"Коммерческая недвижимость Помещение, Иркутск, ...",NaN,0.000000,гостиница,300.000000,NaN,Помещен

Функция, в которой и происходит тестирование.

In [42]:
def make_test():
    """
    :return: DataFrame, содержащий статистику по всем задачам
    """
    test_df = pd.DataFrame(index=result_df.columns)
    test_df['Рейтинг заполненности'] = [0] * len(test_df)
    test_df['Заполнено'] = [0] * len(test_df)
    test_df['Всего'] = [len(result_df)] * len(test_df)
    
    float_null = '0.000000'
    nan = str(float('nan'))
    
    for row in result_df:
        for cell in result_df.loc[:, row]:
            if cell and not cell == float_null and cell != nan and cell != 'НетДанных':  # Проверка на неподходящие значения
                test_df.loc[row, 'Заполнено'] += 1
                
    for row in test_df.iterrows():
        test_df.loc[row[0], 'Рейтинг заполненности'] = str(row[1]['Заполнено'] * 100 // row[1]['Всего']) + '%'
        
    return test_df

In [43]:
test = make_test()

Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [44]:
test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

,Рейтинг заполненности,Заполнено,Всего
Код,100%,15,15
Ссылка,100%,15,15
АдресAhunter,86%,13,15
АктуальнаяСсылкаИсточника,0%,0,15
ДатаПересмотраЭкспозиции,100%,15,15
ДатаПроверкиАктуальности,100%,15,15
ДатаРазмещения,100%,15,15
Город,86%,13,15
Описание,100%,15,15
Подсегмент,93%,14,15


## Для тестов


In [ ]:
query = '''
SELECT
      *
FROM [rway].[Документ].[Задание]
WHERE


'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df